Name: Karel Mackenzie Chandra
Student ID:30373867

In [1]:
# Libraries Used
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pygeohash as pgh

In [2]:
topic_name = 'EventProducer01'

In [3]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('[Demo] Spark Streaming from Kafka into MongoDB')
    .getOrCreate()
)

In [4]:
# Open connection with kafka
topic_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', 'localhost:9092')
    .option('subscribe', topic_name)
    .load()
)

In [5]:
topic_stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# Change the output format so it is much more suitable
output_stream_df = (
    topic_stream_df
    .select(                                      # 1
        split(
            topic_stream_df.value.cast('string'),
            ','
        )
        .alias('data')
    )

)

In [7]:
output_stream_df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: string (containsNull = true)



class DbWriter:
    """# called at the start of processing each partition in each output micro-batch
    def open(self, partition_id, epoch_id):
        self.mongo_client = MongoClient(
            host='localhost',
            port=27017
        )
        self.db = self.mongo_client['fit3182_assignment_db']
        return True
    
    # called once per row of the result dataframe
    # the current code DOES NOT handle duplicate processing
    #   e.g., query fails and restarts just before current micro-batch was fully inserted
    def process(self, row):
        self.db[topic_name].insert_one(row.asDict())"""
        
    def process_batch(batch_df, batch_id):
        pre_data = []
        raw_data = batch_df.collect()
        for l in raw_data:
            pre_data.append(l.asDict())  
        return pre_data
    def logic(pre_data):
        for l in pre_data:
            if l['producer_num'] == 'Producer01':
                print("True")
            else:
                print("False")
    
    """# called once all rows have been processed (possibly with error)
    def close(self, err):
        self.mongo_client.close()"""

In [17]:
# Initialize the Logial Requirement of the assignment
def process_batch(batch_df, batch_id):
    pre_data = []
    raw_data = batch_df.collect()
    for l in raw_data:
        pre_data.append(l.asDict())
        print(pre_data)
    for i in pre_data:
        if i['data'][9] == 'Producer01':
            location1 = (i['data'][0], i['data'][1])
            location2 = (i++['data'][0], i++['data'][1])
            pgh(location1,location2)
            def open(self, partition_id, epoch_id):
                self.mongo_client = MongoClient(
                    host='localhost',
                    port=27017
                )
                self.db = self.mongo_client['fit3182_db']
                return True
    
            # called once per row of the result dataframe
            # the current code DOES NOT handle duplicate processing
            #   e.g., query fails and restarts just before current micro-batch was fully inserted
            def process(self, row):
                data = json.loads(row.value)
        
                db_record = {}
                db_record['_id'] = data.get('bay_id')
                db_record['latitude'] = data.get('latitude')
                db_record['longitude'] = data.get('longitude')
                db_record['air_temperature_celcius'] = data.get('air_temperature_celcius')
                print(db_record)
                self.db[topic].replace_one({'_id': data.get('bay_id')}, db_record, upsert=True)
    
            # called once all rows have been processed (possibly with error)
            def close(self, err):
                self.mongo_client.close()
        else:
            return

{'data': ["{'latitude': '-36.404'", " 'longitude': '142.5467'", " 'air_temperature_celcius': '25'", " 'relative_humidity': '60.9'", " 'windspeed_knots': '6.3'", " 'max_wind_speed': '8.9'", " 'precipitation ': ' 0.00I'", " 'GHI_w/m2': '195'", " 'date': '2022-03-13'", " 'producer_num': 'Producer01'}"]}


In [18]:
# Entry point of the stream
db_writer = (
    output_stream_df
    .writeStream
    .outputMode('append')
    .trigger(processingTime='10 seconds')
    .foreachBatch(process_batch)
)

In [19]:
# Start of the whole program wait for producer to feed the data
db_writer.start()

False
{'data': ["{'latitude': '-37.954'", " 'longitude': '143.918'", " 'air_temperature_celcius': '20'", " 'relative_humidity': '59.7'", " 'windspeed_knots': '8.6'", " 'max_wind_speed': '15.9'", " 'precipitation ': ' 0.01G'", " 'GHI_w/m2': '158'", " 'date': '2022-03-14'", " 'producer_num': 'Producer01'}"]}
["{'latitude': '-37.954'", " 'longitude': '143.918'", " 'air_temperature_celcius': '20'", " 'relative_humidity': '59.7'", " 'windspeed_knots': '8.6'", " 'max_wind_speed': '15.9'", " 'precipitation ': ' 0.01G'", " 'GHI_w/m2': '158'", " 'date': '2022-03-14'", " 'producer_num': 'Producer01'}"]
False
["{'latitude': '-37.858'", " 'longitude': '143.428'", " 'air_temperature_celcius': '14'", " 'relative_humidity': '49.5'", " 'windspeed_knots': '6'", " 'max_wind_speed': '12'", " 'precipitation ': ' 0.00G'", " 'GHI_w/m2': '121'", " 'date': '2022-03-15'", " 'producer_num': 'Producer01'}"]
{'data': ["{'latitude': '-37.858'", " 'longitude': '143.428'", " 'air_temperature_celcius': '14'", " 'rela